In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import os
import pandas as pd
from torchvision.io import read_image
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import Circle
import utils
import networks

# Load data set

In [25]:
# define the train and val splits
train_split = 0.80
test_split = 1 - train_split
noise_levels = [0.1,0.3,0.5,0.7,0.9,1.1]
noise_levels = [0.1,0.3]
model_types = ['CustomCNN3','CustomCNN3']
for model_type in model_types:
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )

    for noise_lvl in noise_levels:
        '''Load data set with noise'''
        #add gassian noise to training data
        data_set = utils.CustomImageDataset(annotations_file="./data/annotations.csv",img_dir="./data",transform=transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            utils.AddGaussianNoise(mean = 0, std = noise_lvl)]))

        # set number of training samples
        numTrainingSamples = int(len(data_set) * train_split)
        # set number of test samples
        numTestSamples = int(len(data_set) - numTrainingSamples)

        # draw test data from data using same seed 42
        (trainData, testData) = random_split(data_set,
            [numTrainingSamples, numTestSamples],
            generator=torch.Generator().manual_seed(42))
        
        '''Load trained model according to noise'''
        if model_type == 'CustomCNN3':
            model = networks.CustomCNN3().to(device)
            checkpoint = torch.load(f'./models/CustomCNN3_state_noise_lvl_{noise_lvl}.pt')
        elif model_type == 'CustomCNN5':
            model = networks.CustomCNN5().to(device)
            checkpoint = torch.load(f'./models/CustomCNN5_state_noise_lvl_{noise_lvl}.pt')
        else: print("Not valid model selected")
        
        
        model.load_state_dict(checkpoint)

        '''Prediction'''
        test_dataloader = DataLoader(testData, batch_size=1, shuffle=False)
        size = len(test_dataloader.dataset)
        num_batches = len(test_dataloader)
        predictions = np.zeros([size,3])
        labels = np.zeros([size,3])
        # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
        # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
        with torch.no_grad():
            i = 0
            for X, label in test_dataloader:
                
                X = X.to(device)
                labels[i,:]=label
                label = label.to(device)
                pred = torch.squeeze(model(X))
                predictions[i,:]=(pred.numpy())
                i += 1 
        '''Export as .csv'''
        predictions = np.round(predictions)
        predictions_df = pd.DataFrame({'image':testData.indices,'x': labels[:,0],'y': labels[:,1], 'r':labels[:,2],'noise':noise_lvl,'x_hat': predictions[:,0],'y_hat':predictions[:,1], 'r_hat':predictions[:,2]})
        predictions_df.tail()
        predictions_df.to_csv(f'predictions_{model_type}_state_noise_lvl_{noise_lvl}.csv',index=False)